In [1]:
import dash
# import dash_core_components as dcc
from dash import dcc
# import dash_html_components as html
from dash import html
import dash_bootstrap_components as dbc

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import date

In [3]:
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MATERIA])

IMPORT INCIDENT TICKETS

In [4]:
# Create initial Dataframe
df_incident = pd.read_csv('/Users/noel.c.joaquin/Desktop/BASE_DATA/INC_YTD_nov2022.csv')
# df_incident.info()

In [5]:
# Replace NaN values with '01-01-2200' 
df_incident['resolved_at'].fillna('01/01/2200', inplace=True)
df_incident['closed_at'].fillna('01/01/2200', inplace=True)
# df_incident.info()

In [6]:
# Save new Dataframe 
df_incident.to_csv('/Users/noel.c.joaquin/Desktop/Python/INC_YTD_nov2022.csv', index=False)

In [7]:
# Recreate Dataframe and conver Date columns to Datetime format
d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

df_incident = pd.read_csv('/Users/noel.c.joaquin/Desktop/Python/INC_YTD_nov2022.csv', parse_dates=['opened_at', 'closed_at', 'resolved_at'],
                            date_parser = d_parser , encoding = 'unicode_escape')
                            
# df_incident.head()

C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_20504\1158756277.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_20504\1158756277.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_20504\1158756277.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')


IMPORT TASK TICKETS

In [8]:
# Create initial Dataframe
df_task = pd.read_csv('/Users/noel.c.joaquin/Desktop/BASE_DATA/TASK_YTD_nov2022.csv')
df_task.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   number     64 non-null     object
 1   state      64 non-null     object
 2   opened_at  64 non-null     object
 3   closed_at  59 non-null     object
dtypes: object(4)
memory usage: 2.1+ KB


In [9]:
# Fill any null values
df_task['closed_at'].fillna('01/01/2200', inplace=True)
# df_task.info()

In [10]:
# Save new Dataframe 
df_task.to_csv('/Users/noel.c.joaquin/Desktop/Python/TASK_YTD_nov2022.csv', index=False)

In [31]:
# Create SCTASK DataFrame
d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

df_task = pd.read_csv('/Users/noel.c.joaquin/Desktop/Python/TASK_YTD_nov2022.csv', parse_dates=['opened_at', 'closed_at'], date_parser = d_parser, encoding='unicode_escape')

df_task.head()

C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_20504\3827722699.py:2: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_20504\3827722699.py:2: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



,number,state,opened_at,closed_at
0,SCTASK1234577,Closed Complete,2022-01-01,2022-02-01
1,SCTASK1234578,Closed Complete,2022-01-02,2022-02-02
2,SCTASK1234580,Closed Complete,2022-01-04,2022-02-04
3,SCTASK1234581,Closed Complete,2022-01-05,2022-02-05
4,SCTASK1234582,Closed Complete,2022-02-01,2022-03-01


GLOBAL VARIABLES

In [12]:
month_date_today = date.today()
split_month_date_today = str(month_date_today).split('-')
month_first_day = str(month_date_today).replace(str(f"-{split_month_date_today[-1]}"), str(f"-01")) 

CARD COUNTS - INC Opened and Resolved

In [40]:
def ticket_count_opened(timeline, df):    
    count = 0

    if timeline == 'current-month':
        date_filter = (df['opened_at'] >= month_first_day)
        count = len(date_filter.loc[date_filter])
        return count
    elif timeline == 'last-month':
        return count
    elif timeline == 'current-year':
        year_first_day = month_first_day.replace(str(f"-{split_month_date_today[1]}"), str(f"-01"))
        count = df.loc[df['opened_at'] >= year_first_day].shape[0]
        return count
    elif timeline == 'ytd':
        count = df.shape[0]
        return count

def ticket_count_resolved(timeline, state, df):    
    count = 0

    if timeline == 'current-month':
        date_filter = (df[state] >= month_first_day) & (df[state] < '2200-01-01')
        count = len(date_filter.loc[date_filter])
        return count
    elif timeline == 'last-month':
        return count
    elif timeline == 'current-year':
        year_first_day = month_first_day.replace(str(f"-{split_month_date_today[1]}"), str(f"-01"))
        date_filter = (df[state] >= year_first_day) & (df[state] < '2200-1-1')
        count = df.loc[date_filter].shape[0]
        return count
    elif timeline == 'ytd':
        date_filter = (df[state] < '2200-1-1')
        count = df.loc[date_filter].shape[0]
        return count

In [35]:
inc_count_opened_cm = ticket_count_opened('current-month', df_incident)
inc_count_opened_lm = ticket_count_opened('last-month', df_incident)
inc_count_opened_cy = ticket_count_opened('current-year', df_incident)
inc_count_opened_ytd = ticket_count_opened('ytd', df_incident)

task_count_opened_cm = ticket_count_opened('current-month', df_task)
task_count_opened_lm = ticket_count_opened('last-month', df_task)
task_count_opened_cy = ticket_count_opened('current-year', df_task)
task_count_opened_ytd = ticket_count_opened('ytd', df_task)

print(f"inc_count_opened_cm: {inc_count_opened_cm}")
print(f"inc_count_opened_lm: {inc_count_opened_lm}")
print(f"inc_count_opened_cy: {inc_count_opened_cy}")
print(f"inc_count_opened_ytd: {inc_count_opened_ytd}")
print("-----------------------")
print(f"task_count_opened_cm: {task_count_opened_cm}")
print(f"task_count_opened_lm: {task_count_opened_lm}")
print(f"task_count_opened_cy: {task_count_opened_cy}")
print(f"task_count_opened_ytd: {task_count_opened_ytd}")

inc_count_opened_cm: 9
inc_count_opened_lm: 0
inc_count_opened_cy: 43
inc_count_opened_ytd: 83
-----------------------
task_count_opened_cm: 5
task_count_opened_lm: 0
task_count_opened_cy: 30
task_count_opened_ytd: 64


In [41]:
inc_count_resolved_cm = ticket_count_resolved('current-month', 'resolved_at', df_incident)
inc_count_resolved_lm = ticket_count_resolved('last-month', 'resolved_at', df_incident)
inc_count_resolved_cy = ticket_count_resolved('current-year', 'resolved_at', df_incident) 
inc_count_resolved_ytd = ticket_count_resolved('ytd', 'resolved_at', df_incident) 


task_count_resolved_cm = ticket_count_resolved('current-month', 'closed_at', df_task)
task_count_resolved_lm = ticket_count_resolved('last-month', 'closed_at', df_task)
task_count_resolved_cy = ticket_count_resolved('current-year', 'closed_at', df_task)
task_count_resolved_ytd = ticket_count_resolved('ytd', 'closed_at', df_task)

print(f"inc_count_resolved_cm: {inc_count_resolved_cm}")
print(f"inc_count_resolved_lm: {inc_count_resolved_lm}")
print(f"inc_count_resolved_cy: {inc_count_resolved_cy}")
print(f"inc_count_resolved_ytd: {inc_count_resolved_ytd}")
print("-----------------------")
print(f"task_count_resolved_cm: {task_count_resolved_cm}")
print(f"task_count_resolved_lm: {task_count_resolved_lm}")
print(f"task_count_resolved_cy: {task_count_resolved_cy}")
print(f"task_count_resolved_ytd: {task_count_resolved_ytd}")

inc_count_resolved_cm: 1
inc_count_resolved_lm: 0
inc_count_resolved_cy: 37
inc_count_resolved_ytd: 75
-----------------------
task_count_resolved_cm: 4
task_count_resolved_lm: 0
task_count_resolved_cy: 28
task_count_resolved_ytd: 59


Volume of Incidents by Issue Type

In [18]:
# Get all 'Research Assessment' types
research_assessment_types = df_incident["u_research_assessment"].unique().tolist()
# print(research_assessment_types)

# Get Count for each 'Research Assessment'
research_assessment_count = []
for issue_type in research_assessment_types:
    count = df_incident.loc[df_incident["u_research_assessment"] == issue_type].shape[0]
    research_assessment_count.append(count)

    print(f"issue_type: {issue_type} - {count}")
# print(research_assessment_count)

# Create DataFrame for Research Assessment Count
df_research_assessment = pd.DataFrame({
    "Research Assessment": research_assessment_types,
    "": research_assessment_count
})

df_research_assessment = df_research_assessment.sort_values('', ascending=False)
df_research_assessment

issue_type: Data Issue - 12
issue_type: Login Issue - 15
issue_type: Product Issue - 12
issue_type: System Issue - 11
issue_type: Workspace Issue - 11
issue_type: Credential Issue - 12
issue_type: nan - 0
issue_type: ESHD Alert - 2
issue_type: Failure > Alert - 2
issue_type: Tableau Issue - 2


,Research Assessment,
1,Login Issue,15
0,Data Issue,12
2,Product Issue,12
5,Credential Issue,12
3,System Issue,11
4,Workspace Issue,11
7,ESHD Alert,2
8,Failure > Alert,2
9,Tableau Issue,2
6,NaN,0


In [19]:
# Make Bar Chart for df_research_assessment
fig = px.bar(df_research_assessment, x="Research Assessment", y="", barmode="group", text_auto=True)
fig.show()

CARD

In [20]:
def card_count(title, title_size, data, data_size):
    card = dbc.Card(
            dbc.CardBody(
                [
                    html.H5(title, className="card-title", style={'font-size': title_size}),
                    html.P(data, style={'font-size': data_size, 'font-weight': 'bold'}),
                ]
            )
        )
    
    return card

card_inc_opened = card_count('Incidents Opened', '1.5rem', inc_opened_cm, '5rem')
card_inc_resolved = card_count('Incidents Resolved', '1.5rem', inc_resolved_cm, '5rem')
card_task_opened = card_count('Incidents Opened', '1.5rem', inc_opened_cm, '5rem')
card_task_resolved = card_count('Incidents Resolved', '1.5rem', inc_resolved_cm, '5rem')

cards = dbc.Row(
        [
            dbc.Col(card_inc_opened, width = 3),
            dbc.Col(card_inc_resolved, width = 3),
            dbc.Col(card_task_opened, width = 3),
            dbc.Col(card_task_resolved, width = 3),
        ]
    )


Tickets Opened and Resolved

In [21]:
def year_month_data(selected_year, state, df):

    initial_year_month_list = df[state].dt.strftime('%Y-%m').unique().tolist()
    year_month_list = []

    for x in initial_year_month_list:
        if selected_year in x:
            year_month_list.append(x)

    # Get available month names for selected year
    get_month_names = df.loc[df[state] > selected_year][state].dt.strftime('%b').unique().tolist()

    return year_month_list, get_month_names
    
inc_ym_2022, inc_mn_2022 = year_month_data('2022', 'opened_at', df_incident)
print(inc_ym_2022)
print(inc_mn_2022)

['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11']
['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']


In [22]:
def year_month_count_opened(year_month_list, state, df):
    year_month_count_opened = []

    for get_curr_year_month in year_month_list:

        get_curr_month = get_curr_year_month.split('-')[1]
        get_curr_year = get_curr_year_month.split('-')[0]
        get_next_month = int(get_curr_month) + 1
        get_next_year = int(get_curr_year) + 1

        if get_next_month > 12:
            
            get_next_month = 1;
            get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))
            get_next_year_month = get_next_year_month.replace(str(get_curr_year), str(get_next_year))

            count = df.loc[(df[state] < f"{get_next_year_month}-01") & 
                                    (df[state] >= f"{get_curr_year_month}-01")]

            year_month_count_opened.append(count.shape[0])
        else:
            get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))

            count = df.loc[(df[state] < f"{get_next_year_month}-01") & 
                                    (df[state] >= f"{get_curr_year_month}-01")]

            year_month_count_opened.append(count.shape[0])

    return year_month_count_opened

inc_ymc_opened_2022 = year_month_count_opened(inc_ym_2022, 'opened_at' ,df_incident)
inc_ymc_resolved_2022 = year_month_count_opened(inc_ym_2022, 'resolved_at' ,df_incident)

task_ymc_opened_2022 = year_month_count_opened(inc_ym_2022, 'opened_at' ,df_task)
task_ymc_resolved_2022 = year_month_count_opened(inc_ym_2022, 'closed_at' ,df_task)

print(f"inc_ymc_opened_2022: {inc_ymc_opened_2022}")
print(f"inc_ymc_resolved_2022: {inc_ymc_resolved_2022}")
print(f"task_ymc_opened_2022: {task_ymc_opened_2022}")
print(f"task_ymc_resolved_2022 : {task_ymc_resolved_2022}")


inc_ymc_opened_2022: [5, 3, 4, 2, 4, 3, 2, 4, 3, 4, 9]
inc_ymc_resolved_2022: [7, 3, 3, 3, 3, 2, 4, 3, 4, 4, 1]
task_ymc_opened_2022: [4, 3, 2, 2, 3, 1, 2, 2, 2, 4, 5]
task_ymc_resolved_2022 : [1, 6, 3, 1, 3, 2, 1, 3, 2, 2, 4]


In [23]:
# Create DataFrame for 'opened_at' column based on 'year_month_list' and 'year_month_count'
df_inc_open_and_close = pd.DataFrame({
    "month_year": inc_ym_2022,
    "count_open": inc_ymc_opened_2022,
    "count_resolved": inc_ymc_resolved_2022
})

df_inc_open_and_close

,month_year,count_open,count_resolved
0,2022-01,5,7
1,2022-02,3,3
2,2022-03,4,3
3,2022-04,2,3
4,2022-05,4,3
5,2022-06,3,2
6,2022-07,2,4
7,2022-08,4,3
8,2022-09,3,4
9,2022-10,4,4


In [24]:
# Create figure for 'df_inc_open_and_close'
colors = px.colors.qualitative.Set3

fig2 = go.Figure(data=[
    go.Bar(name='Opened', x=inc_mn_2022, y=inc_ymc_opened_2022, text=inc_ymc_opened_2022, marker=dict(color=colors[4])),
    go.Bar(name='Resolved', x=inc_mn_2022, y=inc_ymc_resolved_2022, text=inc_ymc_resolved_2022, marker=dict(color=colors[3])),
])

fig2.show()

INC & TASK OPENED

In [25]:
# # Create DataFrame
df_inc_and_task_open = pd.DataFrame({
    "month_year": inc_ym_2022,
    "count_open_inc": inc_ymc_opened_2022,
    "count_open_task": task_ymc_opened_2022,
})

print(df_inc_and_task_open)

   month_year  count_open_inc  count_open_task
0     2022-01               5                4
1     2022-02               3                3
2     2022-03               4                2
3     2022-04               2                2
4     2022-05               4                3
5     2022-06               3                1
6     2022-07               2                2
7     2022-08               4                2
8     2022-09               3                2
9     2022-10               4                4
10    2022-11               9                5


In [26]:
# # Create figure for 'df_inc_and_task_open'
colors = px.colors.qualitative.Set3

fig3 = go.Figure(data=[
    go.Bar(name='Incidents', x=inc_mn_2022, y=inc_ymc_opened_2022, text=inc_ymc_opened_2022, marker=dict(color=colors[4])),
    go.Bar(name='Tasks', x=inc_mn_2022, y=task_ymc_opened_2022, text=task_ymc_opened_2022, marker=dict(color=colors[3])),
])

fig3

In [27]:
# Filler Pie Chart
df_pie = px.data.tips()
fig_pie = px.pie(df_pie, values='tip', names='day', color='day',
             color_discrete_map={'Thur':'lightcyan',
                                 'Fri':'cyan',
                                 'Sat':'royalblue',
                                 'Sun':'darkblue'})
fig_pie.show()

In [28]:
second_row = dbc.Row([
    dbc.Col(dcc.Graph(figure=fig_pie), width=4),
    dbc.Col(dcc.Graph(figure=fig_pie), width=4),
    dbc.Col([
        dbc.Row(dcc.Graph(figure=fig_pie)),
        dbc.Row(dcc.Graph(figure=fig_pie))
    ], width=4),
])

In [29]:
# Create server variable with Flask server object for use with gunicorn
server = app.server

# Construct App Layout
app.layout = html.Div(children=[
    html.H1('Analytics Platform Ticket Dashboard'),
    dcc.Tabs(id="tabs-content-input", value='current-month', children=[
        dcc.Tab(label='Current Month', value='current-month'),
        dcc.Tab(label='Last Month', value='last-month'),
        dcc.Tab(label='Current Year', value='current-year'),
        dcc.Tab(label='Year to Date', value='ytd')
    ]),
    html.Br(),
    html.Div(id='tabs-content-output'),

])

@app.callback(Output('tabs-content-output', 'children'),
              Input('tabs-content-input', 'value'))
def render_content(tab):
    if tab == 'current-month':
        return html.Div([
            dbc.Row(cards, justify="around"),
            html.Br(),
            dbc.Row(second_row),
        ])
    elif tab == 'last-month':
        return html.Div([
            dbc.Row(cards, justify="around")  
        ])
    elif tab == 'current-year':
        return html.Div([
            dbc.Row(cards, justify="around")  
        ])
    elif tab == 'ytd':
        return html.Div([
            dbc.Row(cards, justify="around")  
        ])

app.run_server()

Dash app running on http://127.0.0.1:8050/
